In [13]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import LSTM
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.regularizers import l2

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[2], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [9]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [11]:
run_experiment(X,y)

Epoch 1/50
8/8 [==============================] - 0s 48ms/step - loss: 0.6926 - accuracy: 0.5761
Epoch 2/50
8/8 [==============================] - 0s 47ms/step - loss: 0.6171 - accuracy: 0.6539
Epoch 3/50
8/8 [==============================] - 0s 48ms/step - loss: 0.5315 - accuracy: 0.7634
Epoch 4/50
8/8 [==============================] - 0s 47ms/step - loss: 0.4663 - accuracy: 0.7930
Epoch 5/50
8/8 [==============================] - 0s 48ms/step - loss: 0.3310 - accuracy: 0.8959
Epoch 6/50
8/8 [==============================] - 0s 46ms/step - loss: 0.2666 - accuracy: 0.9080
Epoch 7/50
8/8 [==============================] - 0s 50ms/step - loss: 0.2056 - accuracy: 0.9321
Epoch 8/50
8/8 [==============================] - 0s 48ms/step - loss: 0.1646 - accuracy: 0.9463
Epoch 9/50
8/8 [==============================] - 0s 48ms/step - loss: 0.1184 - accuracy: 0.9682
Epoch 10/50
8/8 [==============================] - 0s 48ms/step - loss: 0.1108 - accuracy: 0.9639
Epoch 11/50
8/8 [============

8/8 [==============================] - 0s 50ms/step - loss: 0.0073 - accuracy: 0.9989
Epoch 17/50
8/8 [==============================] - 0s 50ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 18/50
8/8 [==============================] - 0s 51ms/step - loss: 0.0048 - accuracy: 0.9989
Epoch 19/50
8/8 [==============================] - 0s 49ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 20/50
8/8 [==============================] - 0s 50ms/step - loss: 0.0059 - accuracy: 0.9989
Epoch 21/50
8/8 [==============================] - 0s 50ms/step - loss: 0.0039 - accuracy: 0.9989
Epoch 22/50
8/8 [==============================] - 0s 53ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 23/50
8/8 [==============================] - 0s 52ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 24/50
8/8 [==============================] - 0s 50ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 25/50
8/8 [==============================] - 0s 50ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 26/50
8/8 [===============

In [14]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3),input_shape=(None,n_length,n_features)))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3)))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_2(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
    
run_experiment_2(X,y)

Epoch 1/50
8/8 [==============================] - 0s 52ms/step - loss: 0.7704 - accuracy: 0.5049
Epoch 2/50
8/8 [==============================] - 0s 50ms/step - loss: 0.6913 - accuracy: 0.5619
Epoch 3/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6276 - accuracy: 0.6429
Epoch 4/50
8/8 [==============================] - 0s 50ms/step - loss: 0.5602 - accuracy: 0.7492
Epoch 5/50
8/8 [==============================] - 0s 48ms/step - loss: 0.4421 - accuracy: 0.8554
Epoch 6/50
8/8 [==============================] - 0s 49ms/step - loss: 0.3606 - accuracy: 0.8719
Epoch 7/50
8/8 [==============================] - 0s 49ms/step - loss: 0.2542 - accuracy: 0.9310
Epoch 8/50
8/8 [==============================] - 0s 48ms/step - loss: 0.1812 - accuracy: 0.9409
Epoch 9/50
8/8 [==============================] - 0s 48ms/step - loss: 0.1685 - accuracy: 0.9485
Epoch 10/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1021 - accuracy: 0.9748
Epoch 11/50
8/8 [============

8/8 [==============================] - 0s 51ms/step - loss: 1.7745e-04 - accuracy: 1.0000
 >#3: 68.122 
Epoch 1/50
8/8 [==============================] - 0s 53ms/step - loss: 0.7414 - accuracy: 0.5246
Epoch 2/50
8/8 [==============================] - 0s 51ms/step - loss: 0.6687 - accuracy: 0.5991
Epoch 3/50
8/8 [==============================] - 0s 50ms/step - loss: 0.5652 - accuracy: 0.7273
Epoch 4/50
8/8 [==============================] - 0s 52ms/step - loss: 0.4359 - accuracy: 0.8445
Epoch 5/50
8/8 [==============================] - 0s 50ms/step - loss: 0.3147 - accuracy: 0.8916
Epoch 6/50
8/8 [==============================] - 0s 52ms/step - loss: 0.2066 - accuracy: 0.9430
Epoch 7/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1486 - accuracy: 0.9573
Epoch 8/50
8/8 [==============================] - 0s 50ms/step - loss: 0.0882 - accuracy: 0.9836
Epoch 9/50
8/8 [==============================] - 0s 52ms/step - loss: 0.0750 - accuracy: 0.9814
Epoch 10/50
8/8 [======

In [15]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu'),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='selu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_3(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
    
run_experiment_3(X,y)

Epoch 1/50
8/8 [==============================] - 0s 51ms/step - loss: 0.7488 - accuracy: 0.5597
Epoch 2/50
8/8 [==============================] - 0s 51ms/step - loss: 0.5901 - accuracy: 0.6813
Epoch 3/50
8/8 [==============================] - 0s 51ms/step - loss: 0.4647 - accuracy: 0.7853
Epoch 4/50
8/8 [==============================] - 0s 51ms/step - loss: 0.3407 - accuracy: 0.8543
Epoch 5/50
8/8 [==============================] - 0s 54ms/step - loss: 0.2542 - accuracy: 0.9080
Epoch 6/50
8/8 [==============================] - 0s 52ms/step - loss: 0.1848 - accuracy: 0.9288
Epoch 7/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1409 - accuracy: 0.9463
Epoch 8/50
8/8 [==============================] - 0s 53ms/step - loss: 0.0927 - accuracy: 0.9671
Epoch 9/50
8/8 [==============================] - 0s 53ms/step - loss: 0.0894 - accuracy: 0.9660
Epoch 10/50
8/8 [==============================] - 0s 51ms/step - loss: 0.0803 - accuracy: 0.9748
Epoch 11/50
8/8 [============

8/8 [==============================] - 0s 50ms/step - loss: 2.3099e-04 - accuracy: 1.0000
 >#3: 58.515 
Epoch 1/50
8/8 [==============================] - 0s 51ms/step - loss: 0.7658 - accuracy: 0.5630
Epoch 2/50
8/8 [==============================] - 0s 50ms/step - loss: 0.5917 - accuracy: 0.6681
Epoch 3/50
8/8 [==============================] - 0s 52ms/step - loss: 0.4771 - accuracy: 0.7755
Epoch 4/50
8/8 [==============================] - 0s 52ms/step - loss: 0.3754 - accuracy: 0.8335
Epoch 5/50
8/8 [==============================] - 0s 52ms/step - loss: 0.2423 - accuracy: 0.9036
Epoch 6/50
8/8 [==============================] - 0s 52ms/step - loss: 0.1789 - accuracy: 0.9321
Epoch 7/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1631 - accuracy: 0.9430
Epoch 8/50
8/8 [==============================] - 0s 51ms/step - loss: 0.0937 - accuracy: 0.9704
Epoch 9/50
8/8 [==============================] - 0s 50ms/step - loss: 0.0584 - accuracy: 0.9858
Epoch 10/50
8/8 [======